Datastory 2

Groep 96

Rama Pamudji - 11170220

Lucas Belderink - 12151750

Wordcount = 550

<h1>Lonen oneerlijk verdeeld over posities</h1>

<h3>Inleiding</h3>

Het komt als geen verrassing aan wanneer verteld wordt dat professionele voetbalspelers een grote som geld verdienen. Aan de lonen van de bekendste voetballers is te merken dat voetballers meer verdienen naarmate zij ook beter presteren. Er is hier dus te zien dat er een groot verschil zit in de lonen van voetbalspelers gebaseerd op algemene vaardigheden. Toch is er een groter verschil te zien in de lonen van spelers gebaseerd op de positie van de spelers. Ik ben daarom van mening dat de lonen ongelijk verdeeld zijn over de verschillende posities op het veld.


In [1128]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import ipywidgets as widgets

In [1129]:
df = pd.read_csv('fifa_data.csv', usecols=['Wage', 'Position', 'Overall'])

In [1130]:
# Verander monetaire waarden (die letters bevatten die grote getallen representeren) naar float waarden.
def money_to_num(string):
    num = str(string)
    if 'K' not in num and 'M' not in num and '€' not in num and '$' not in num:
        return float(num)
    
    num = num[1:]
    
    if num.endswith('K'):
        num = float(num[0:-1]) * 1000  
    elif num.endswith('M'):
        num = float(num[0:-1]) * 1000000
    else:
        num = float(num)
    
    return num

In [1131]:
df1 = df.copy()
df1['Wage'] = df1['Wage'].apply(money_to_num)

# De wages (beloningen) en overall skill worden gegroepeerd en gemiddeld genomen over de verschillende posities
df1 = df1.groupby('Position').mean().round(0)

In [1132]:
fig1 = px.scatter(df1, title='Average wage and overall skill per position', 
                  x='Wage', 
                  y='Overall', 
                  size='Wage', 
                  color=df1.index)

fig1.update_layout(yaxis_tickformat='€')
fig1.show()

<h3>Verschil</h3>

Om de verschillen tussen de lonen te zien van de verschillende posities moet er gekeken worden naar de gemiddelde lonen van de verschillende posities. In de grafiek is te zien dat desondanks de RAM en LAM positie dezelfde gemiddelde algemene vaardigheden hebben, de LAM positie alsnog een €7.000 hogere gemiddelde loon heeft. Dit geldt nog zwaarder voor de RF en LF positie, waarbij de RF positie ondanks zijn lagere algemene vaardigheden alsnog een €8.000 hogere gemiddelde loon heeft dan de LF positie. Er is dus in zekere hoogte een verschil te zien in de gemiddelde lonen tussen de verschillende posities.

In [1133]:
wage_df = []

# Nieuwe dataframe met als index de posities, kolommen de bepaalde overall skill waarden en als value de gemiddelde beloning
for i in range(60, 93):
    
    # Alle rijen met dezelfde overall skill worden meegenomen
    df2 = df.loc[df['Overall'] == i].copy() 
    
    df2['Wage'] = df2['Wage'].apply(money_to_num)
    
    wage = df2.groupby(['Position'])['Wage'].mean().round(2)
    
    temp_df = pd.DataFrame({'Position': [x for x in wage.index], 
                        i: [x for x in wage],}).set_index('Position')
    
    wage_df.append(temp_df)

wage_df = pd.concat(wage_df, axis=1)

In [1134]:
def update_plot(x):
    
    fig = px.bar(wage_df, y=x, color=x, title='Average wage per position per overall skill')
    fig.update_xaxes(title='Position', tickangle=45)
    fig.update_yaxes(title='Wage')
    fig.show()


x = widgets.IntSlider(min=60, max=92, 
                        step=1, 
                        description='Overall skill', 
                        continuous_update=True, 
                        size=100)

x.style.handle_color = 'orange'

widgets.interactive(update_plot, x=x)

interactive(children=(IntSlider(value=60, description='Overall skill', max=92, min=60, style=SliderStyle(handl…

<h3>Verschil van lonen per algemene vaardigheid punt</h3>

Verder moet er dieper worden gekeken naar de verschillende overall skill points, oftewel algemene vaardigheden. Daarom is bij bovenstaande grafiek een schuifregelaar toegepast om de gemiddelde lonen te zien per gemiddelde algemene vaardigheid punt. Zo heeft de LWB positie bij een vaardigheid punt van 73 minimaal een €12.000 hogere loon dan de andere posities. Dit is bij een hogere vaardigheid punt van 84 nog duidelijker te zien waarbij de RW positie maar liefst een €150.000 hogere loon heeft dan de LB positie. Er is in de grafieken aan de gemiddelde lonen per positie te zien dat er een ongelijke verdeling van loon is over de verschillende posities.

In [1135]:
fifadata = pd.read_csv('fifa_data.csv')

In [1136]:
fifadata['Wage'] = fifadata['Wage'].apply(money_to_num)

In [1137]:
valuequant = pd.cut(fifadata["Wage"], 3, labels = ["Low", "Medium", "High"])
valuequantq = pd.qcut(fifadata["Wage"], 3, labels = ["Low", "Medium", "High"])

valuequanthist = go.Figure(data = [go.Histogram(x = valuequant)])
valuequanthistq = go.Figure(data = [go.Histogram(x = valuequantq)])

In [1138]:
overallquant = pd.cut(fifadata["Overall"], 3, labels = ["Low", "Medium", "High"])
overallquantq = pd.qcut(fifadata["Overall"], 3, labels = ["Low", "Medium", "High"])

overallquanthist = go.Figure(data = [go.Histogram(x = overallquant)])
overallquanthistq = go.Figure(data = [go.Histogram(x = overallquantq)])

In [1139]:
valuedim = go.parcats.Dimension(
    values = valuequantq,
    label = "Wage"
)

abilitydim = go.parcats.Dimension(
    values = overallquantq,
    label = "Ability",
)

positiondim = go.parcats.Dimension(
    values = fifadata['Position'],
    label = "Position",
)

# category codes opstellen voor de colorscale
valuequantq.astype("category").cat.codes

color = valuequantq.astype("category").cat.codes
colorscale = [[0, 'purple'], [0.5, 'green'], [1, 'red']];

parc = go.Parcats(
    dimensions=[valuedim,
                abilitydim,
                positiondim
               ],
    # freeform laat de gebruiker de quantielen zelf van volgorde veranderen door ze te slepen
    arrangement = "freeform",
    line={'color': color, 'colorscale': colorscale}
)

parfig = go.Figure(data = parc)

parfig.update_layout(
    height = 1000
)

parfig.show()

<h3>Loon gebaseerd op vaardigheden</h3>

Toch is er te debatteren dat de lonen bepaald worden door de vaardigheden van de spelers. In bovenstaande grafiek is te zien dat de rode strepen vanaf hoge lonen voornamelijk lopen door de hoge vaardigheden. Er kan hier worden geconcludeerd dat spelers dus op vaardigheden worden beloond. Toch is er te zien dat er alsnog lage en medium lonen bewegen naar hoge vaardigheden. Ook is er te zien dat de middel vaardigheden erg verdeeld zijn door de hoge, medium en lage lonen, deze zijn verspreid over de verschillende posities. Uit de grafiek is dus te zien dat posities ondanks de vaardigheden van de spelers alsnog oneerlijk beloond worden.


<h3>Conclusie</h3>

Professionele voetbalspelers hebben relatief hoge lonen, maar deze lonen zijn oneerlijk verdeeld over de posities. Aan de hand van de eerste grafiek was te merken dat ondanks dezelfde vaardigheid punt twee verschillende posities alsnog een groot verschillende loon hadden. Dit werd nog duidelijker in de tweede grafiek waarbij per vaardigheid punt gezien kon worden dat er een oneerlijke verdeling was over de verschillende posities. In de derde grafiek was te zien dat er hogere vaardigheid punten hoger beloond werden, maar bij de medium vaardigheid punten was alsnog te zien dat een groot deel van de posities minder werd beloond dan de andere posities. Al met al, op basis van positie worden de lonen van voetbalspelers oneerlijk verdeeld.
